# **ME 193, Spring 2021**
## Final Project Notebook 


**Notes**:

1. 

In [2]:
import os
import numpy as np
import pandas as pd
import requests
from IPython.display import JSON

import sklearn.tree as tree
import sklearn.metrics as mt
import sklearn.ensemble as ens
import matplotlib.pyplot as plt
import sklearn.model_selection

import seaborn as sns
from sklearn.cluster import KMeans
from scipy.sparse import csr_matrix
from matplotlib.colors import ListedColormap
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

## Import data DO NOT RUN
Scrape vivino website by analyzing Network tab on the all search and pulling API request info. id = 1 is red wines and id = 2 is whites. Parems for search included
Red network url: https://www.vivino.com/api/explore/explore?country_code=US&currency_code=USD&grape_filter=varietal&min_rating=1&order_by=ratings_average&order=desc&page=1&price_range_max=500&price_range_min=0&wine_type_ids[]=1

In [3]:
url = "https://www.vivino.com/api/explore/explore"
headers= {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"}
#page_num = 1
matches = []

# loops through pages; ~40,000 wines, 25 per page -> ~1600 pages; but sample less
for i, page_num in enumerate(range(1000, 1650)):
    r = requests.get(url,
        params = {
            "country_code": "US",
            "currency_code":"USD",
            "grape_filter":"varietal",
            "min_rating":"1",
            "order_by":"ratings_average",
            "order":"desc",
            "page": page_num,
            "price_range_max":"450",
            "price_range_min":"0",
            "wine_type_ids[]":"1"
        }, headers=headers)
    print(page_num, end='\r')
    reds = r.json()
    matches.extend(reds["explore_vintage"]["matches"])
print('data aquired')

KeyboardInterrupt: 

In [ ]:
results = [
    (
        t["vintage"]["wine"]["winery"]["name"], 
        #f'{t["vintage"]["wine"]["name"]} {t["vintage"]["year"]}',
        t["vintage"]["wine"]["name"],
        t["price"]["amount"],
        t["vintage"]["year"],
        t["vintage"]["wine"]["style"]["varietal_name"] if t["vintage"]["wine"]["style"] != None else float("NaN"),
        t["vintage"]["wine"]["region"]["name"],
        t["vintage"]["wine"]["region"]["country"]["name"],
        t["vintage"]["statistics"]["ratings_average"],
        t["vintage"]["wine"]["taste"]["structure"]["acidity"] if t["vintage"]["wine"]["taste"]["structure"] != None else float("NaN"),
        t["vintage"]["wine"]["taste"]["structure"]["sweetness"] if t["vintage"]["wine"]["taste"]["structure"] != None else float("NaN"),
        t["vintage"]["wine"]["taste"]["structure"]["tannin"] if t["vintage"]["wine"]["taste"]["structure"] != None else float("NaN"),
        # flavors
        [j["group"] for j in t["vintage"]["wine"]["taste"]["flavor"]] if t["vintage"]["wine"]["taste"]["flavor"] != None else float("NaN")
    ) for t in matches
]

df_reds2 = pd.DataFrame(results,columns=['Winery','Wine','Price','Year','Grape','Region','Country','Rating','Acidity','Sweetness','Tannin','Flavor'])
#df_reds = pd.DataFrame(results,columns=['Winery','Wine','Price','Year','Region','Country','Rating','Acidity','Sweetness','Tannin','Flavor'])
print(page_num)
print(len(df_reds2))
df_reds2.head()
#df_reds.to_pickle("./reds.pkl")  # saves the database

In [ ]:
print(len(df_reds2))
df_reds = df_reds.append(df_reds2, ignore_index=True)
print((df_reds.iloc[1000:1100]))
df_reds.head()
df_reds.to_pickle("./reds.pkl")  # saves the database

In [ ]:
print(len(df_reds))
df_reds.to_pickle("./reds.pkl")  # saves the database

## Load created database with line below START HERE

In [ ]:
df_reds = pd.read_pickle("./reds.pkl")
print(len(df_reds))
df_reds.head()

## Data Exploration

In [ ]:
ratings = df_reds[['Rating', 'Grape', 'Country']]

plt.rc("font", size=15)
ratings.Rating.value_counts(sort=False).plot(kind='bar')
plt.title('Rating Distribution\n')
plt.xlabel('Rating')
plt.ylabel('Count');

In [ ]:
plt.rc("font", size=15)
ratings.Country.value_counts(sort=False).plot(kind='bar')
plt.title('Country Distribution\n')
plt.xlabel('Country of Production')
plt.ylabel('Count');

In [4]:
plt.rc("font", size=15)
ratings.Grape.value_counts(sort=False).plot(kind='bar')

plt.figure(figsize=(10, 8))        
plt.title('Grape Distribution\n')
plt.xlabel('Grape Type')
plt.ylabel('Count');

NameError: name 'ratings' is not defined

In [ ]:
# Define Function to calculate Within Cluster Variance
def withinClusVar(X,centers,labels,labelset):
    variance = []
    for i in labelset:
        variance.append(np.linalg.norm(X[labels==i]-centers[i])**2/len(X[labels==i]))
    return sum(variance)

In [ ]:
X_test_wine = df_reds.drop(['Winery', 'Wine', 'Grape', 'Region', 'Country', 'Flavor'], axis=1)
X_test_wine["Year"] = pd.to_numeric(X_test_wine["Year"], errors='coerce')
X_test_wine = X_test_wine.dropna()
n_clusters = np.linspace(2,50,49, dtype='int')
withinClusterVar = []

for n in n_clusters:
    kmeans = KMeans(n_clusters=n, n_init = 10, max_iter=200).fit(X_test_wine)
    centers = kmeans.cluster_centers_
    labels = kmeans.fit_predict(X_test_wine)
    labelset = np.unique(labels)
    
    withinClusterVar.append(withinClusVar(X_test_wine, centers, labels, labelset))

plt.figure(figsize = (12,9)) 
plt.plot(n_clusters, withinClusterVar)
plt.title('Within Cluster Variance for each Cluster')
plt.xlabel('Cluster')
plt.ylabel('WCV')

In [ ]:
minWCV = min(withinClusterVar)
indexWCV = withinClusterVar.index(minWCV)
minCluster = n_clusters[indexWCV]
print("The K Value that minimizes the Within Cluster Variance is: K =",minCluster)


In [ ]:
kmeans = KMeans(n_clusters = 2, n_init = 10, max_iter = 300).fit(X_test_wine)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

plt.figure(figsize = (12,9)) 
sns.scatterplot(data=df_reds, x="Price", y="Rating", hue="Year", marker = 'o', legend = 'auto', palette = 'winter')
plt.xlabel('Price', fontsize = '16')
plt.ylabel('Rating',fontsize = '16')
plt.title('WINES', fontsize = '22');